# Введение в MapReduce модель на Python


In [29]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [30]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [31]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [32]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [33]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [34]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [35]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [36]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [37]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [38]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [39]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [40]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [41]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [42]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [43]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.6212433820794945),
 (1, 2.6212433820794945),
 (2, 2.6212433820794945),
 (3, 2.6212433820794945),
 (4, 2.6212433820794945)]

## Inverted index

In [44]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [45]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [46]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [47]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('it', 18)]),
 (1, [('banana', 2), ('is', 18), ('what', 10)])]

## TeraSort

In [48]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.030430854279746344),
   (None, 0.1113406822637738),
   (None, 0.16793977392021198),
   (None, 0.20162284890381232),
   (None, 0.21229748730627507),
   (None, 0.25086359566913363),
   (None, 0.26199230849960053),
   (None, 0.27076655207230604),
   (None, 0.3199314119798793),
   (None, 0.3370229100608666),
   (None, 0.3668251369579286),
   (None, 0.42562237822902405),
   (None, 0.48329625326989445),
   (None, 0.4852932146456421)]),
 (1,
  [(None, 0.548647842938865),
   (None, 0.6313288514832232),
   (None, 0.6754322673707592),
   (None, 0.6785947691598672),
   (None, 0.7127852174298365),
   (None, 0.7255290700615515),
   (None, 0.7357145939761911),
   (None, 0.7415067965392654),
   (None, 0.7508163949365072),
   (None, 0.7891653561886671),
   (None, 0.8116191846096701),
   (None, 0.8464491830997559),
   (None, 0.8974373081678174),
   (None, 0.9124830595857999),
   (None, 0.9879529249880912),
   (None, 0.994485384118021)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [71]:
def MAP(_, row: NamedTuple): #ключ группировки для всех пользователей
    yield (0, row)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

#поиск пользователя с максимальным числом контактов
def REDUCE(_, rows: Iterator[NamedTuple]):
    max_cont = 0
    max_cont_user = None
    for row in rows:
        if row.social_contacts > max_cont:
            max_cont = row.social_contacts
            max_cont_user = row
    yield max_cont_user


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[User(id=3, age=33, social_contacts=800, gender='female')]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [69]:
def MAPG(_, row: NamedTuple):
    yield (0, row)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

#подсчет среднего значения контактов
def REDUCEG(_, rows: Iterator[NamedTuple]):
    count = len(rows)
    sum = 0
    for i in rows:
        sum += i.social_contacts
    yield sum / count

output = MapReduce(RECORDREADER, MAPG, REDUCEG)
output = list(output)
output

[390.0]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [73]:
def group(iterable):
  p = {}
  for (k2, v2) in iterable:
    p[k2] = sorted(p.get(k2, []) + [v2])
  return p.items()

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print(f"max: {output}")

output = MapReduce(RECORDREADER, MAPG, REDUCEG)
output = list(output)
print(f"average: {output}")

max: [User(id=3, age=33, social_contacts=800, gender='female')]
average: [390.0]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [76]:
from typing import List

def MAP(key: int, value: int):
  yield (value, value)

def RECORDREADER():
  data = [(1, 111), (2, 222), (3, 22), (4, 222), (5, 555), (6, 666), (7, 66), (8, 666), (9, 999), (10, 99)]
  return data

def REDUCE(key: int, row: List[int]):
  yield key

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(sorted(output))
output

[22, 66, 99, 111, 222, 555, 666, 999]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [77]:
def MAP_SELECT(_, row: NamedTuple):
    if row.gender == "female":
        yield (row, row)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

def REDUCE_SELECT(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)


output = MapReduce(RECORDREADER, MAP_SELECT, REDUCE_SELECT)
output = list(output)
output

[(User(id=1, age=25, social_contacts=240, gender='female'),
  [User(id=1, age=25, social_contacts=240, gender='female')]),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  [User(id=2, age=25, social_contacts=500, gender='female')]),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  [User(id=3, age=33, social_contacts=800, gender='female')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [82]:
class UserProj(NamedTuple):
  id: int
  social_contacts: int

input_collect = [
    User(id=0, age=59, gender='male', social_contacts=110),
    User(id=0, age=21, gender='female', social_contacts=180),
    User(id=1, age=25, gender='female', social_contacts=220),
    User(id=2, age=25, gender='female', social_contacts=220),
    User(id=3, age=31, gender='female', social_contacts=90),
    User(id=4, age=23, gender='male', social_contacts=720)
]

def MAP(key, value):
  yield (UserProj(id=value.id, social_contacts=value.social_contacts), UserProj(id=value.id, social_contacts=value.social_contacts))

def RECORDREADER():
  return [(u.id, u) for u in input_collect]

def REDUCE(key, rows):
  yield (key, key)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(sorted(output))
output

[(UserProj(id=0, social_contacts=110), UserProj(id=0, social_contacts=110)),
 (UserProj(id=0, social_contacts=180), UserProj(id=0, social_contacts=180)),
 (UserProj(id=1, social_contacts=220), UserProj(id=1, social_contacts=220)),
 (UserProj(id=2, social_contacts=220), UserProj(id=2, social_contacts=220)),
 (UserProj(id=3, social_contacts=90), UserProj(id=3, social_contacts=90)),
 (UserProj(id=4, social_contacts=720), UserProj(id=4, social_contacts=720))]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [88]:
input_collect2 = [
    User(id=2, age=59, gender='male', social_contacts=160),
    User(id=5, age=21, gender='female', social_contacts=150),
    User(id=6, age=25, gender='female', social_contacts=220),
    User(id=2, age=25, gender='female', social_contacts=220),
    User(id=3, age=31, gender='female', social_contacts=90),
    User(id=8, age=23, gender='male', social_contacts=760)]

def MAP_UN(_, row: NamedTuple):
    yield (row.id, row)


def RECORDREADER():
    return [(u.id, u) for u in input_collect + input_collect2]

def REDUCE_UN(row: str, rows: Iterator[NamedTuple]):
    yield (rows[0], rows[0])

output = MapReduce(RECORDREADER, MAP_UN, REDUCE_UN)
output = list(output)
output

[(User(id=0, age=59, social_contacts=110, gender='male'),
  User(id=0, age=59, social_contacts=110, gender='male')),
 (User(id=1, age=25, social_contacts=220, gender='female'),
  User(id=1, age=25, social_contacts=220, gender='female')),
 (User(id=2, age=25, social_contacts=220, gender='female'),
  User(id=2, age=25, social_contacts=220, gender='female')),
 (User(id=3, age=31, social_contacts=90, gender='female'),
  User(id=3, age=31, social_contacts=90, gender='female')),
 (User(id=4, age=23, social_contacts=720, gender='male'),
  User(id=4, age=23, social_contacts=720, gender='male')),
 (User(id=5, age=21, social_contacts=150, gender='female'),
  User(id=5, age=21, social_contacts=150, gender='female')),
 (User(id=6, age=25, social_contacts=220, gender='female'),
  User(id=6, age=25, social_contacts=220, gender='female')),
 (User(id=8, age=23, social_contacts=760, gender='male'),
  User(id=8, age=23, social_contacts=760, gender='male'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [87]:
def MAP_INTERSECT(_, row: NamedTuple):
    yield (row.id, row)

def RECORDREADER():
    return [(u.id, u) for u in input_collect + input_collect2]

def REDUCE_INTERSECT(row_id: int, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield rows

output = MapReduce(RECORDREADER, MAP_INTERSECT, REDUCE_INTERSECT)
output = list(output)
output

[[User(id=0, age=59, social_contacts=110, gender='male'),
  User(id=0, age=21, social_contacts=180, gender='female')],
 [User(id=3, age=31, social_contacts=90, gender='female'),
  User(id=3, age=31, social_contacts=90, gender='female')]]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [92]:
def MAP_DIF(key, value):
    if value in input_collect:
        yield (value, 0)
    elif value in input_collect2:
        yield (value, 1)

def RECORDREADER():
  return [(u.id, u) for u in input_collect + input_collect2]

def REDUCE_DIF(key, rows):
  if (rows == [0]):
    yield (key, key)

output = MapReduce(RECORDREADER, MAP_DIF, REDUCE_DIF)
output = list(sorted(output))
output

[(User(id=0, age=21, social_contacts=180, gender='female'),
  User(id=0, age=21, social_contacts=180, gender='female')),
 (User(id=0, age=59, social_contacts=110, gender='male'),
  User(id=0, age=59, social_contacts=110, gender='male')),
 (User(id=1, age=25, social_contacts=220, gender='female'),
  User(id=1, age=25, social_contacts=220, gender='female')),
 (User(id=4, age=23, social_contacts=720, gender='male'),
  User(id=4, age=23, social_contacts=720, gender='male'))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [96]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str
    dep_id: int

users_collection = [
    User(id=1, age=59, gender="male", social_contacts=20, dep_id=1),
    User(id=2, age=21, gender="male", social_contacts=240, dep_id=2),
    User(id=3, age=25, gender="male", social_contacts=642, dep_id=3),
    User(id=4, age=25, gender="female", social_contacts=123, dep_id=3),
    User(id=4, age=31, gender="female", social_contacts=247, dep_id=4),
]


class Department(NamedTuple):
    id: int
    name: str

dep_collection = [
    Department(id=1, name="First"),
    Department(id=2, name="Second"),
    Department(id=3, name="Third"),
    Department(id=4, name="Fourth"),
    Department(id=5, name="Fifth"),
]


def MAP_JOIN(dep_id, row):
    yield (dep_id, row)


def REDUCE_JOIN(dep_id, rows):
    users = []
    dep = None

    for i in rows:
        if type(id) is User:
            users += [i]
        else:
            dep = i

    for i in rows:
        if type(i) is User:
            yield (i, i.dep_id, dep)


def RECORDREADER():
    return [(user.dep_id, user) for user in users_collection] + [
        (dep.id, dep) for dep in dep_collection
    ]


output = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
output = list(output)
joined_data = output
joined_data

[(User(id=1, age=59, social_contacts=20, gender='male', dep_id=1),
  1,
  Department(id=1, name='First')),
 (User(id=2, age=21, social_contacts=240, gender='male', dep_id=2),
  2,
  Department(id=2, name='Second')),
 (User(id=3, age=25, social_contacts=642, gender='male', dep_id=3),
  3,
  Department(id=3, name='Third')),
 (User(id=4, age=25, social_contacts=123, gender='female', dep_id=3),
  3,
  Department(id=3, name='Third')),
 (User(id=4, age=31, social_contacts=247, gender='female', dep_id=4),
  4,
  Department(id=4, name='Fourth'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [98]:
def MAP_GROUP(dep_id, user, dep):
    yield (dep_id, user)

def RECORDREADER():
    return [(dep_id, user, dep) for user, dep_id, dep in joined_data]

def REDUCE_GROUP(dep_id, rows):
    yield f"departament with id={dep_id} has {len(rows)} user(s)"


output = MapReduce(RECORDREADER, MAP_GROUP, REDUCE_GROUP)
output = list(output)
output

['departament with id=1 has 1 user(s)',
 'departament with id=2 has 1 user(s)',
 'departament with id=3 has 2 user(s)',
 'departament with id=4 has 1 user(s)']

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [50]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [101]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(I):
    yield ((i, k), small_mat[i, j] * w)

def REDUCE(key, values):
  (i, k) = key
  yield ((i, k), sum(values))

Проверьте своё решение

In [102]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [103]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [117]:
def RECORDREADER():
    for i in range(I):
        for j in range(J):
            for k in range(K):
                yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

def MAP(el1, el2):
    (i, j), v1 = el1
    (j, k), v2 = el2
    yield ((i, k), v1 * v2)

def REDUCE(key, values):
    (i, k) = key
    k3 = (i, k)
    v3 = 0
    for j in range(J):
        v3 += values[j]
    yield (k3, v3)


I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

In [118]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)


def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for (i, k), vw in reduce_output:
        mat[i, k] = vw
    return mat


np.allclose(reference_solution, asmatrix(solution))  # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [123]:
def INPUT():
    global maps

    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))
    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))

def MAP(el1, el2):
    (i, j), v1 = el1
    (j, k), v2 = el2
    yield ((i, k), v1 * v2)


def REDUCE(key, values):
    (i, k) = key
    k3 = (i, k)
    v3 = 0
    for j in range(J):
        v3 += values[j]
    yield (k3, v3)

maps = 2
reducers = 2

part_output = MapReduceDistributed(INPUT, MAP, REDUCE, COMBINER=None)
part_output = [
    (part_id, list(part)) for (part_id, part) in part_output
]

sol = []
for i in part_output:
    for j in i[1]:
        sol += [j]

print(asmatrix(sol))

240 key-value pairs were sent over a network.
[[0.69792427 1.36557892 0.41856322 1.21343496 0.37609335 0.40538706
  1.58099323 0.55625595 0.88760699 1.47751362 0.88181815 0.77143137
  1.48698175 0.26660258 0.6429685  1.20062572 0.6478613  1.1574849
  1.53614588 0.72428355 1.39852734 0.22627578 1.43092428 1.35829214
  0.43891267 1.10600074 0.34848812 1.52428526 1.22334593 0.92315738
  1.03312499 0.81833538 0.91099988 0.77365016 0.3699906  0.81375256
  0.89646497 1.52848413 1.42004445 1.17941469]
 [0.39271522 0.9863226  0.28773277 0.84046523 0.20681998 0.19822427
  1.06363725 0.3854604  0.65479397 1.02898133 0.70655606 0.53850175
  1.04017528 0.21339549 0.49388274 0.71075906 0.33584742 0.79010274
  1.04531894 0.39969101 0.96156775 0.17945166 0.92434089 0.99157071
  0.2755461  0.76035778 0.27365963 1.01491104 0.77521455 0.49747015
  0.59189435 0.50705116 0.47476044 0.34921076 0.2273626  0.48075697
  0.50906573 1.04132199 1.02240504 0.8936577 ]]


In [124]:
np.allclose(reference_solution, asmatrix(solution))  # should return true

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

Обобщая предыдущие решения, стоит отметить, что решение будет работать, так как каждая пара элементов (генерируемая при помощи RECORDREADER) идентифицируется по индексам. При этом могут быть сгенерированы случайные пожмножества элементов матрицы.